In [ ]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.1 MB/s eta 0:00:00


In [ ]:
!pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 4.4 MB/s eta 0:00:00


In [ ]:
!pip install lmqg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.7/71.7 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.6/662.6 kB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 68.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.5

In [ ]:
!pip install python-docx

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 99.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.6 MB/s eta 0:00:00


In [ ]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.2 MB/s eta 0:00:00


**1) Question Paper Generation**

In [2]:
import docx
import random
import os
from transformers import T5Tokenizer, T5ForConditionalGeneration
from lmqg import TransformersQG

# Constants
NUM_2_MARK_QUESTIONS = 6
NUM_5_MARK_QUESTIONS = 2
NUM_7_MARK_QUESTIONS = 4
MODEL_NAME = "t5-base"
MAX_LENGTH = 512

def read_docx(docx_file):
    """Reads a DOCX file and returns the text content."""
    try:
        doc = docx.Document(docx_file)
    except Exception as e:
        print(f"Error reading the file: {e}")
        return None

    text_content = ""
    for para in doc.paragraphs:
        text_content += para.text
    return text_content

def generate_questions(num_questions, text_content, model, tokenizer):
    """Generates a specified number of questions."""
    # Split the text_content into chunks of MAX_LENGTH
    text_chunks = [text_content[i:i+MAX_LENGTH] for i in range(0, len(text_content), MAX_LENGTH)]

    questions = []
    for text_chunk in text_chunks:
        question_answer = model.generate_qa(text_chunk, num_questions=num_questions)
        chunk_questions = [question for question, _ in question_answer]
        questions.extend(chunk_questions)

    return questions

def main(docx_file, updated_question_paper_file):
    """Main function to generate the question paper."""
    text_content = read_docx(docx_file)

    if text_content is not None:
        try:
            model = TransformersQG('lmqg/t5-base-squad-qag')
            tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
        except RuntimeError as e:
            print(f"Error loading the model: {e}")
            return None

        questions_2_mark = generate_questions(NUM_2_MARK_QUESTIONS, text_content, model, tokenizer)
        questions_5_mark = generate_questions(NUM_5_MARK_QUESTIONS, text_content, model, tokenizer)
        questions_7_mark = generate_questions(NUM_7_MARK_QUESTIONS, text_content, model, tokenizer)

        doc = docx.Document()
        doc.add_heading('2 Marks Questions', level=1)
        for i, question in enumerate(questions_2_mark, 1):
            doc.add_paragraph(f'{i}. {question}')

        doc.add_heading('5 Marks Questions', level=1)
        for i, question in enumerate(questions_5_mark, 1):
            doc.add_paragraph(f'{i}. {question}')

        doc.add_heading('7 Marks Questions', level=1)
        for i, question in enumerate(questions_7_mark, 1):
            doc.add_paragraph(f'{i}. {question}')

        doc.save(updated_question_paper_file)

if __name__ == "__main__":
    docx_file = "/content/con.docx"  # create a file with some content like book as docx
    updated_question_paper_file = "/content/qp.docx"  #create a empty file to get output as docx
    main(docx_file, updated_question_paper_file)



#After running this part check the folders for qp.docx and your question paper is ready there

ModuleNotFoundError: ignored

**2) Question Bank Generation**

In [ ]:
!pip install pprint

In [ ]:
import PyPDF2
import pprint
from lmqg import TransformersQG

# Initialize model
model = TransformersQG('lmqg/t5-base-squad-qag')

# Open the PDF file that has some content regarding any thing in few pages
with open('Google LLC.pdf', 'rb') as f:
    pdf_reader = PyPDF2.PdfReader(f)

    # Extract text from all pages
    context = ''
    for page in pdf_reader.pages:
        context += page.extract_text()

# Split the input text into multiple parts if it exceeds the maximum length
max_length = 512
if len(model.tokenizer.encode(context)) > max_length:
    split_context = []
    start_index = 0
    end_index = max_length
    while end_index <= len(context):
        split_context.append(context[start_index:end_index])
        start_index = end_index
        end_index += max_length
else:
    split_context = [context]

# Generate questions for each part of the input text
questions_list = []
for part in split_context:
    question_answer = model.generate_qa(part)
    questions = [question for question, _ in question_answer]
    questions_list.extend(questions)

# Print the generated questions
for question in questions_list:
    pprint.pprint(question)

In [1]:
# Filter out duplicate questions
unique_questions = list(set(questions_list))

# Print the unique questions
for question in unique_questions:
    pprint.pprint(question)



#Finally the Questions for question bank will be generated

NameError: ignored